На основании базы знаний с ПРАВИЛАМИ СТРАХОВАНИЯ ОТВЕТСТВЕННОСТИ АЭРОПОРТОВ И АВИАЦИОННЫХ ТОВАРОПРОИЗВОДИТЕЛЕЙ создайте нейро-консультанта, который бы отвечал на вопросы клиентов по информации, содержащейся в представленном документе. Напишите промпт в system и user самостоятельно. Добейтесь того, чтобы нейро-консультант в своих ответах не ссылался на документы, переданные ему для ответа.

Документ с базой знаний можно найти тут: https://docs.google.com/document/d/11MU3SnVbwL_rM-5fIC14Lc3XnbAV4rY1Zd_kpcMuH4Y

In [1]:
!pip install -q langchain_openai==0.1.22 faiss-cpu==1.8.0 openai==1.55.3 tiktoken==0.7.0 langchain-core==0.2.33 langchain==0.2.14 langchain_community==0.2.12 langchain-text-splitters==0.2.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.6/389.6 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 391.5/391.5 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.8/311.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 1.9 MB/s eta 0:00:00


In [2]:
import requests
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
import tiktoken
import matplotlib.pyplot as plt
import getpass
import os
import openai
from openai import OpenAI
from langchain.vectorstores import FAISS
import re
import getpass
from google.colab import userdata
from langchain_openai import ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter, MarkdownHeaderTextSplitter, CharacterTextSplitter
from langchain.docstore.document import Document

In [3]:
from google.colab import userdata

# Использование ключа API от ProxyAPI
key = userdata.get('PROXY_API_KEY')
os.environ["OPENAI_API_KEY"] = key

# Адрес сервера ProxyAPI
base_url = 'https://api.proxyapi.ru/openai/v1'
os.environ["OPENAI_BASE_URL"] = base_url

# Создание клиента OpenAI с использованием API ключа из переменных среды
client = OpenAI()

In [4]:
# функция для загрузки документа по ссылке из гугл драйв
def load_document_text(url: str) -> str:
    # Extract the document ID from the URL
    match_ = re.search('/document/d/([a-zA-Z0-9-_]+)', url)
    if match_ is None:
        raise ValueError('Invalid Google Docs URL')
    doc_id = match_.group(1)

    # Download the document as plain text
    response = requests.get(f'https://docs.google.com/document/d/{doc_id}/export?format=txt')
    response.raise_for_status()
    text = response.text

    return text

In [5]:
# База знаний, которая будет подаваться в langChain
data_from_url= load_document_text('https://docs.google.com/document/d/11MU3SnVbwL_rM-5fIC14Lc3XnbAV4rY1Zd_kpcMuH4Y')

In [6]:
system = load_document_text('https://docs.google.com/document/d/1pe8yUe2kScsfxgPKFRbcBKOfXwS7b5rM8NjQDVVdEY0/edit?usp=sharing')

In [7]:
system

'\ufeffТы - консультант компании страховании ответсвенности аэропортов и авиационных товаропроизводителей, ответь на вопрос сотрудника компании на основе документа с информацией. Не придумывай ничего от себя, отвечай максимально по документу. А также нельзя ссылаться в ответах на документы с информацией.'

In [9]:
source_chunks=[]
splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=0)       # заполните нужными значениями, проведите эксперименты
for chunk in splitter.split_text(data_from_url):
    source_chunks.append(Document(page_content=chunk, metadata={}))
print("Общее количество чанков: ", len(source_chunks))

Общее количество чанков:  117


In [10]:
# Инициализирум модель эмбеддингов
embeddings = OpenAIEmbeddings()

# Создадим индексную базу из разделенных фрагментов текста
db = FAISS.from_documents(source_chunks, embeddings)

In [14]:
# Функция, которая позволяет выводить ответ модели в удобочитаемом виде
def insert_newlines(text: str, max_len: int = 170) -> str:
    words = text.split()
    lines = []
    current_line = ""
    for word in words:
        if len(current_line + " " + word) > max_len:
            lines.append(current_line)
            current_line = ""
        current_line += " " + word
    lines.append(current_line)
    return " ".join(lines)

def answer_index(system, topic, search_index, temp=1, verbose=1):

    # Поиск релевантных отрезков из базы знаний
    docs = search_index.similarity_search(topic, k=5)
    if verbose: print('\n ===========================================: ')
    message_content = re.sub(r'\n{2}', ' ', '\n '.join([f'\nОтрывок документа №{i+1}\n=====================' + doc.page_content + '\n' for i, doc in enumerate(docs)]))
    if verbose: print('message_content :\n ======================================== \n', message_content)
    client = OpenAI()
    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": f"Ответь на вопрос сотрудника на основе представленной информации. Не упоминай документ с информацией для ответа сотруднику в ответе. Документ с информацией для ответа сотруднику: {message_content}\n\nВопрос сотрудника: \n{topic}"}        # заполните нужными значениями, проведите эксперименты
    ]

    if verbose: print('\n ===========================================: ')

    completion = client.chat.completions.create(
        model="gpt-4o-mini",                               # заполните нужным значением
        messages=messages,
        temperature=0.2                           # заполните нужными значениями, проведите эксперименты
    )
    answer = insert_newlines(completion.choices[0].message.content)
    return answer

Проверьте работу консультанта на самостоятельно сформулированных вопросах.

In [15]:
# вопрос пользователя
topic= "Кто такой изготовитель?"
ans=answer_index(system, topic, db)
ans


 ===========================================: 
message_content :
 
Отрывок документа №1
=====================1.2.14. Производственная деятельность - любые работы и услуги, выполняемые сертифицированными (лицензированными) Изготовителями (Поставщиками), связанные с изготовлением (производством) и/или поставкой, а также обслуживанием, ремонтом, продлением ресурсов и сроков службы, доработкой и модернизацией авиационной и космической техники. 
1.2.15. Разработчик - организация или дееспособные физические лица, зарегистрированные в качестве индивидуальных предпринимателей, производящие проектную, расчетную, конструкторскую, эксплуатационную и иную документацию для реализации производства, испытаний и эксплуатации авиационной и космической техники, имеющие соответствующие лицензии и сертификаты на право заниматься указанной деятельностью, выданные им соответствующими уполномоченными на то органами государственного регулирования, осуществляющие авторский контроль и несущие имущественную отв

' Изготовитель – это организация или дееспособные физические лица, зарегистрированные в качестве индивидуальных предпринимателей, которые производят авиационную и  космическую технику. Они должны иметь сертифицированное (лицензированное) производство с одобренной системой качества продукции и сертификатом типа продукции, выданными  соответствующими уполномоченными органами государственного регулирования. Изготовители также осуществляют авторский контроль и несут имущественную ответственность за  свою продукцию и услуги по закону, а также реализуют произведенную технику.'

In [16]:
# вопрос пользователя
topic= "Что такое субъекты страхования?"
ans=answer_index(system, topic, db)
ans


 ===========================================: 
message_content :
 
Отрывок документа №1
=====================1.2.16. Третьи лица - физические или юридические лица, муниципальные образования, субъекты Российской Федерации, Российская Федерация, не имеющие договорных отношений со Страхователем, имеющие право на предъявление претензий к Страхователю о возмещении причиненного им вреда. К третьим лицам не относится сам Страхователь/ Застрахованные лица, их работники, во время выполнения ими своих служебных обязанностей, а также физические или юридические лица, выступающие от имени Страхователя. 
2. СУБЪЕКТЫ СТРАХОВАНИЯ 
2.1. Страхователями признаются юридические или физические лица, являющиеся владельцами или эксплуатантами аэропортов, поставщиками авиационных товаров и услуг, а также органы управления воздушным движением. 
2.2. Страховщик – страховая организация АО «АльфаСтрахование», действующая в соответствии с законодательством РФ.  
Отрывок документа №2
=====================2.3. Догов

' Субъектами страхования признаются юридические или физические лица, являющиеся владельцами или эксплуатантами аэропортов, поставщиками авиационных товаров и услуг, а  также органы управления воздушным движением.'